# Bacteria-phage co-evolution animation

This notebook generates frames to make a gif or movie visualizing a simulation of bacteria interacting with phages. 

In [ ]:
# import packages
import numpy as np
from matplotlib import pyplot as plt
import os, glob
import pandas as pd
import matplotlib.cm as cm
from scipy import sparse

Note to self: check out [this method of doing animations?](https://matplotlib.org/2.1.2/gallery/animation/dynamic_image2.html)

Name ideas:
`bubblEvolve`, `bubble race`, `bubbly`

In [ ]:
def load_simulation(folder, timestamp):
    """
    Load the simulation output files into the format that will be used to create the movie.
    
    Inputs:
    folder : relative path to folder containing data files
    timestamp : timestamp suffix on all simulation files (i.e. 2019-02-11T14:17:12.365722), 
                    generated with datetime.datetime.now().isoformat()
    
    Returns:
    f : chemostat flow rate
    c0 : initial nutrient concentration
    g : bacteria growth rate
    B : phage burst size
    R : bacteria spacer loss rate
    eta : probability of spacer acquisition following bacterial escape from phage
    pv : probability of phage infection success if bacterium doesn't have matching spacer
    alpha : rate of phage adsorption to bacteria 
    e : relative benefit provided by a matching CRISPR spacer - if e = 0, no benefit, if e = 1, phages never successfully infect
    L : protospacer length (number of nucleotides)
    mu : phage mutation rate per nucleotide
    m_init : number of initial phage clones (m_init = 1 here)
    pop_array : numpy array saved in sparse format with all population data at saved simulation time points.
    
    pop_array structure:
        
    | Columns                 | Description                                                       |
    | 0                       | nB0 : total number of bacteria without spacers                    |
    | 1 : max_m + 1`          | nBi : number of bacteria with each spacer type from 1 to max_m    |
    | max_m + 1 : 2*max_m + 1 | nVi : number of phages with each protospacer type from 1 to max_m |
    | 2*max_m + 2 or -2       | C : total units of nutrients available                            |
    | 2*max_m + 3 or -1       | t (mins) : time in minutes since the start of the simulation      |
    
    
    max_m : total number of unique protospacer sequences that exist during the simulation 
    mutation_times : list of times at which each mutation happens in the simulation. The total list
                      is the same length as all_phages - each entry in each sublist is the times at which a mutant 
                      of that phage type appeared. 
    parent_list : list of same shape as mutation_times that identifies the source phage sequence for each phage
                      mutant that appears in the simulation. For example, parent_list[1][0] = 0, meaning that the 
                      phage sequence identified as type 1 first arose by mutation from type 0. The corresponding mutation time is
                      mutation_times[1][0] = 1.6400876.
    all_phages : list of same length as the outer list of mutation_times and parent_list identifying the nucleotide sequences
                      of each phage that ever existed in the simulation. The index of a phage in all_phages corresponds to its
                      position in pop_array.
    """
    
    # load parameters
    parameters = pd.read_csv(folder + "/parameters_%s.txt" %timestamp, delimiter = '\t', header=None)
    parameters.columns = ['parameter', 'value']
    parameters.set_index('parameter')

    f = float(parameters.loc[parameters['parameter'] == 'f']['value'])
    c0 = float(parameters.loc[parameters['parameter'] == 'c0']['value'])
    g = float(parameters.loc[parameters['parameter'] == 'g']['value'])
    B = float(parameters.loc[parameters['parameter'] == 'B']['value'])
    R = float(parameters.loc[parameters['parameter'] == 'R']['value'])
    eta = float(parameters.loc[parameters['parameter'] == 'eta']['value'])
    pv = float(parameters.loc[parameters['parameter'] == 'pv']['value'])
    alpha = float(parameters.loc[parameters['parameter'] == 'alpha']['value'])
    e = float(parameters.loc[parameters['parameter'] == 'e']['value'])
    L = float(parameters.loc[parameters['parameter'] == 'L']['value'])
    mu = float(parameters.loc[parameters['parameter'] == 'mu']['value'])
    m_init = float(parameters.loc[parameters['parameter'] == 'm_init']['value'])

    # load list of all phages that ever existed
    with open(folder + "/all_phages_%s.txt" %timestamp, "rb") as all_phages_file:
        all_phages = all_phages_file.readlines()
    
    #print("creating list of all phages")
    all_phages = recreate_phage(all_phages[0])

    # load simulation population data
    pop_array = sparse.load_npz(folder + "/pop_array_%s.txt.npz" %timestamp) # fast <3 <3 <3
    max_m = int((pop_array.shape[1] -3)/2)
    pop_array = pop_array.toarray() # convert to dense array

    # load list of mutation times
    with open("%s/mutation_times_%s.txt" %(folder,timestamp), "rb") as mut_f:
        mutation_t = mut_f.readlines()

    mutation_times = recreate_parent_list(mutation_t[0])

    # load parent list
    with open("%s/parents_%s.txt" %(folder,timestamp), "rb") as par:
        parents = par.readlines()
    parent_list = recreate_parent_list(parents[0])
    
    return f, c0, g, B, R, eta, pv, alpha, e, L, mu, m_init, pop_array, max_m, mutation_times, parent_list, all_phages


In [ ]:
def recreate_phage(phage_row):
    """
    Used in load_simulation to get a list of all unique phages that have existed in the simulation.
    
    Input: the list of prophages for a particular timepoint (i.e phage[-1])
    Output: the same line formatted as a list of lists of integers
    """
    phage_list = []
    
    phages_string = phage_row.decode("utf-8").split('[')[2:]
    for phage in phages_string:
        phage = phage.split(']')[0]
        phage_list.append(list(np.array(phage.split(','),dtype=int)))
    
    return phage_list

In [ ]:
def recreate_parent_list(parent_row):
    """
    Input: list of prophage parents for a particular time point (i.e. parent_list[-1], where parent_list is read
    in above)
    
    Output: the same line formatted as a list of tuples of integers, or 'nan' if the phage has no parent (i.e. 
    is one of the original phages)
    """
    parent_list_row = []
    parents_string = parent_row.decode("utf-8").split('[')[2:]
    for parent in parents_string:
        parent = parent.split(']')[0]
        if parent == "''": # this is one of the original phages with no back mutations
            parent_list_row.append([])
        else: # has at some point arisen by mutation
            # check if any of the list are blank
            parent = parent.split(',')
            try:
                ind = parent.index("''")
                parent[ind] = np.nan
            except:
                pass
            try:
                ind = parent.index('')
                parent[ind] = np.nan
            except:
                pass
            parent_list_row.append(list(np.array(parent,dtype='float32')))
        
    return parent_list_row

In [ ]:
def find_nearest(array, value):
    """
    Find the index `idx` of `array` that gives `array[idx]` nearest to `value`.    
    """
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
# data location
folder = "data"
timestamp = "2019-02-11T14:17:12.365722"

In [ ]:
# load simulation data
f, c0, g, B, R, eta, pv, alpha, e, L, mu, m_init, pop_array, \
    max_m, mutation_times, parent_list, all_phages = load_simulation(folder, timestamp)

In [ ]:
# create distance matrix for all the phages that ever lived in the simulation
# calculate the string distance between each pairwise combination of phage sequences and store in a matrix

all_phages = np.array(all_phages)

distance_matrix = np.zeros((len(all_phages), len(all_phages)))

for i in range(len(all_phages)):
    distance_matrix[i] = np.sum(np.abs(all_phages - all_phages[i]), axis = 1)

In [ ]:
# create dictionary that for each phage identifies its parent phage, at what times it arose by mutation, 
# which nucleotide in the parent sequence mutated, and how far away its parent is from the original phage sequence(s).

# the initial phage sequence is at position 0 in all_phages

m_init = int(m_init)
    
phages_dict = {}

for i, phage in enumerate(all_phages):
    if i < m_init:
        continue
    
    parent_ids = parent_list[i]
    
    parent_distances = []
    mutation_positions = []
    angle = []
    full_distance = []
    mutation_ts = mutation_times[i]
    angles = []
    total_distances = []
    
    parent_angles = []
    
    if len(np.unique(parent_ids)) > 1:
        loop_list = parent_ids
        
    else:
        loop_list = np.unique(parent_ids)
        mutation_ts = [mutation_ts[0]]
        
    for j, pid in enumerate(loop_list):
        pid = int(pid)
        mutation_t = mutation_ts[j]
        parent_distance = distance_matrix[i, pid]
        parent_distances.append(parent_distance)
        
        mutation_pos = np.where(np.abs(phage - np.array(all_phages[pid])) == 1)[0]
        mutation_positions.append(mutation_pos)
          
    phages_dict[i] = {"sequence": phage, "parents": loop_list, "mutation_times": mutation_ts, "parent_distance": parent_distances,
                    "mutation_position": mutation_positions}

## Make frames for bubble plot

In [ ]:
# create directory to store image frames
!mkdir frames/

In [ ]:
# delete previous frames
!rm frames/*

In [ ]:
scale = 80. # parameter to scale bubble size - make it bigger or smaller to adjust bubble size
spread = 50. # how much all the possible mutants from a phage spread out in angle in the plot
length = 1. # scale for how far away each concentric ring of bubbles is
box_size = 10.1 # relative figure size
transparency = 0.05 # alpha value for connecting lines 
colours1 = cm.gist_rainbow(np.linspace(0,1,30)) # colormap for bubble colours

# population size legend
legend_alignment = 'bottom left'
legend_spacing = 0.13

if legend_alignment == 'bottom left':
    y_offset = 0.3
    x_offset = 0.1

# timing information   
skip = 20 # only plot every skip generations
end_time = 8000 # if the total simulation runs longer than end_time generations, stop the movie at end_time

# where to plot the initial phage sequence
center_position_x = 0
center_position_y = 0

# legend parameters
shift = 1 # offset for legend pop size bubble
legend_sizes = np.logspace(0,4,5) # legend population sizes

# loop through simulation and create frames
for i in range(int((pop_array[-1,-1]*g*c0)/skip)):
    time = i*skip
    
    if time > end_time:
        break
        
    t_ind = find_nearest(pop_array[:,-1], time/(g*c0))

    fig, axs = plt.subplots(1,2, figsize = (15, 8))

    ax0 = axs[1]
    ax1 = axs[0]

    # set figure properties: set size and remove ticks
    ax0.set_xlim(-box_size,box_size)
    ax0.set_ylim(-box_size,box_size)
    ax0.set_xticks([])
    ax0.set_yticks([])

    ax1.set_xlim(-box_size,box_size)
    ax1.set_ylim(-box_size,box_size)

    ax1.set_xticks([])
    ax1.set_yticks([])
    
    # plot the initial phage sequence
    ax0.scatter(center_position_x, center_position_y, s = scale*np.log(pop_array[t_ind, max_m+1]+1), 
                c = ['k'], edgecolors = 'k')
    ax1.scatter(center_position_x, center_position_y, s = scale*np.log(pop_array[t_ind, 1]+1), 
                c = ['k'], edgecolors = 'k')

    # legend 
    ax1.annotate("Population size:", (-box_size + x_offset*box_size - 0.5, 
                                0-box_size*(y_offset - 0.1)), fontsize = 14)
    for l in range(len(legend_sizes)):
        ax1.scatter(-box_size + x_offset*box_size, 0-box_size*y_offset - box_size*legend_spacing*l, c = ['none'], 
                    s = scale*np.log(shift+legend_sizes[l]), edgecolors = 'k')
        ax1.annotate(str(round(legend_sizes[l],3)), (-box_size + (x_offset + 0.1)*box_size, 
                        0-box_size*(y_offset + 0.02) - box_size*legend_spacing*l), fontsize = 14)
    
    # plot circles
    for key, value in phages_dict.items():

        # size of each bacteria and phage sub-population
        size_phage = pop_array[t_ind, max_m + 1 + key]
        size_bac = pop_array[t_ind, 1 + key]

        if size_phage > 0 or size_bac > 0: # only determine plotting coordinates if population exists

            shortest_distance = np.min(distance_matrix[key, :int(m_init)]) # fewest number of mutations from initial phage to this phage
            angle_chain = []
            total_distance = 0

            phage = key
            parent_phage = phage
            time2 = time

            while parent_phage > m_init-1: # continue following back the history of each population
                mutation_ts = phages_dict[phage]["mutation_times"] # get all the mutations that have happened
                possible_parent_inds = np.where(np.array(mutation_ts) < time2/(g*c0))[0] # get all the indices for mutations before time time2
                possible_parent_phages, sortinds = np.unique(np.array(phages_dict[phage]["parents"])[possible_parent_inds], return_index = True) # unique parent phages

                possible_parent_phages = np.array(possible_parent_phages, dtype = 'int')[np.argsort(sortinds)] # get possible parents in the order in which they happened
                #parent_pop_sizes = pop_array[t_ind, max_m +1 : 2*max_m + 1][possible_parent_phages] # get phage populations for parents
                #parent_phage = possible_parent_phages[np.nonzero(parent_pop_sizes)[0][0]]
                parent_phage = possible_parent_phages[0]
                parent_ind = np.sort(sortinds)[0]
                distance_from_parent = phages_dict[phage]["parent_distance"][parent_ind]
                mutation_pos = phages_dict[phage]["mutation_position"][parent_ind]
                mutation_pos = mutation_pos[0] # in case of double mutant - this is crude, FIX THIS
                angle_chain.append(mutation_pos)
                total_distance += distance_from_parent
                phage = parent_phage
                time2 = mutation_ts[int(possible_parent_inds[-1])]*g*c0
                initial_phage_parent = parent_phage

            mutation_ts = phages_dict[key]["mutation_times"]
            possible_parent_inds = np.where(np.array(mutation_ts) < time/(g*c0))[0] # get all the indices for mutations before time time2
            possible_parent_phages, sortinds = np.unique(np.array(phages_dict[key]["parents"])[possible_parent_inds], return_index = True) # unique parent phages
            possible_parent_phages = np.array(possible_parent_phages, dtype = 'int')[np.argsort(sortinds)] # get possible parents in the order in which they happened

            parent_phage = possible_parent_phages[0]
            parent_ind = np.sort(sortinds)[0]
            mutation_pos = phages_dict[key]["mutation_position"][parent_ind]
            mutation_pos = mutation_pos[0] # in case of double mutant - this is crude, FIX THIS

            if len(angle_chain) < 2:
                parent_angle = 0
                angle = angle_chain[-1] * (360/L) * np.pi / 180

            else:
                # calculate angle from angle_chain
                parent_angle = angle_chain[-1] * (360/L) * np.pi / 180
                angle = parent_angle - (spread/2 )* np.pi / 180 + angle_chain[-2]*(spread/30)*np.pi / 180 

                for a in angle_chain[::-1][2:]:
                    parent_angle = angle
                    angle = parent_angle - (spread/2 )* np.pi / 180 + a*(spread/30)*np.pi / 180 
                    #print(parent_angle, angle)

                    
            if parent_angle == 0:
                colour = colours1[mutation_pos]
            else:
                colour_centre = parent_angle/(2*np.pi)
                colours2 = cm.gist_rainbow(np.linspace(colour_centre-0.5/total_distance,colour_centre+0.5/total_distance,30))
                colour = colours2[mutation_pos]
            
            initial_phage_pos_x = center_position_x
            initial_phage_pos_y = center_position_y

            ax0.scatter(length*total_distance*np.cos(angle) + initial_phage_pos_x, 
                        length*total_distance*np.sin(angle) + initial_phage_pos_y, s = scale*np.log(size_phage+1),
                           c = [colour], edgecolors='k')

            ax1.scatter(length*total_distance*np.cos(angle) + initial_phage_pos_x, 
                        length*total_distance*np.sin(angle) + initial_phage_pos_y, s = scale*np.log(size_bac+1),
                           c = [colour], edgecolors='k')

            # connect each circle to its parent with a line
            ax0.plot([length*(total_distance-1)*np.cos(parent_angle) + initial_phage_pos_x, length*total_distance*np.cos(angle) + initial_phage_pos_x], 
                 [length*(total_distance-1)*np.sin(parent_angle) + initial_phage_pos_y, 
                  length*total_distance*np.sin(angle) + initial_phage_pos_y], 'k-', alpha = transparency)

            if size_bac > 0:        
                ax1.plot([length*(total_distance-1)*np.cos(parent_angle) + initial_phage_pos_x, 
                      length*total_distance*np.cos(angle) + initial_phage_pos_x], 
                     [length*(total_distance-1)*np.sin(parent_angle)+ initial_phage_pos_y, 
                      length*total_distance*np.sin(angle)+ initial_phage_pos_y],
                         'k-', alpha = transparency)
    
    ax0.set_title("Phages", fontsize = 20)
    ax1.set_title("Bacteria", fontsize = 20)
    fig.suptitle('Time = %s generations' %time, fontsize=16, x = 0.415, y = 0.05, ha = 'left')
    plt.tight_layout(rect = [0,0.06, 1, 0.97])
    plt.savefig("frames/frame_%04d.png" % (i,), dpi = 80)
    plt.close()

In [ ]:
# make gif using ImageMagick

gif_name = 'mutations_gif_%s' %(timestamp)

file_list = glob.glob('frames/*.png') # Get all the pngs in the current directory
list.sort(file_list, key=lambda x: int(x.split('_')[1].split('.png')[0])) # Sort the images by #, this may need to be tweaked for your use case

with open('image_list.txt', 'w') as file:
    for item in file_list:
        file.write("%s\n" % item)

os.system('convert @image_list.txt {}.gif'.format(gif_name))

In [ ]:
# optional: convert gif to movie
# this uses ffmpeg: sudo apt install ffmpeg

os.system('ffmpeg -f gif -i %s.gif %s.mp4' %(gif_name, gif_name))